## Part 0


### Setup a Docker image: install a Data Manipulation framework (PySpark or Pandas or any other that will help answer the challenge) and anything else you need. Include the Dockerfile in the submission.

In [ ]:
import csv 
import json
import os 
import pandas as pd 
import sys 
from itertools import chain

## Part I 

#### Mapping of credit_card_vendor to list of card prefix

In [ ]:
# prefix_vendor = list of credit first digits that are representing this vendor.
maestro = ['5018', '5020', '5038', '56']
mastercard = ['51', '52', '54', '55', '222']
visa = ['4']
amex = ['34', '37']
discover = ['6011', '65']
diners = ['300', '301', '304', '305', '36', '38']
jcb16 = ['35']
jcb15 = ['2131', '1800']

In [ ]:
# Make dictionary of vendors 
vendors = {'maestro' : maestro,
           'mastercard' : mastercard,
           'visa' : visa,
           'amex' : amex,
           'discover' : discover,
           'diners' : diners,
           'jcb16' : jcb16,
           'jcb15' : jcb15}

In [ ]:
# vendors

In [ ]:
# Map all credit first digits to the vendor to a flattened list 
credit_cards = maestro + mastercard + visa + amex + discover + diners + jcb16 + jcb15

In [ ]:
# credit_cards

### Import, load, and return the fraud file

In [ ]:
from experiment import (get_fraud_file)

In [ ]:
test_fraud_file = ('test_datasets/test_fraud')

In [ ]:
from pathlib import Path

file_name = Path(test_fraud_file)
if file_name.exists():
    print("exists") 
else:
    print("does not exist")

In [ ]:
def test_get_fraud_file():
    test_fraud_file = ('test_datasets/test_fraud')
     return fraud_dataset


In [ ]:
fraud_dataset = get_fraud_file(fraud_file)

In [ ]:
len(fraud_dataset)

# Part I

### Sanitize data of both transaction-001.zip and transaction-002.zip by removing transactions where column `credit_card_number` is not part of the previous provided list. For example, a credit card that starts with `98` is not a valid card and should be discarded from the sanitized dataset.


In [ ]:
file_list=['datasets/transaction-001','datasets/transaction-002']

In [ ]:
def load_transaction_files(file_list):
    main_dataframe = pd.DataFrame(pd.read_csv(file_list[0]))
    for i in range(1,len(file_list)):
        data = pd.read_csv(file_list[i])
        df = pd.DataFrame(data)
        main_dataframe = pd.concat([main_dataframe,df],axis=0)
    transaction_files = main_dataframe.reset_index(drop=True)
    return transaction_files

In [ ]:
transaction_files = load_transaction_files(file_list)
transaction_files

#### Sanity check that the files loaded properly and the dataframe is of the correct size 
Also reset the index to show it as one complete file instead of two files each of size 250000 entries. 


In [ ]:
len(transaction_files) # Should be 500,000

In [ ]:
# transaction_files.drop_duplicates()

In [ ]:
def get_prefix_column(transaction_files):
    numbers = transaction_files["credit_card_number"]
    start_nums = []
    for index, number in enumerate(numbers):
        str_num = str(number)
        strt_val = list(filter(str_num.startswith, credit_cards)) 
        start_nums.append(strt_val)
    start_vals = list(chain(*start_nums)) 
    start_vals = [int(i) for i in start_vals]
    return start_vals

start_vals = get_prefix_column(transaction_files)

In [ ]:
def filter_transactions(transaction_files, credit_cards, start_vals):
    """
    Takes in a pandas dataframe and a list of credit card numbers 
    and filters the dataframe to create a new column 
    based on if the credit_card_number starts with one of the 
    valid prefix vendor values 
    """
    numbers = transaction_files["credit_card_number"]
    str_nums = []
    valid_nums = []
    for index, number in enumerate(numbers):
        str_num = str(number)
        result = list(filter(str_num.startswith, credit_cards)) != [] # if the element startswith a credit_cardscredit_card_numbers value keep it, otherwise, remove 
        valid_nums.append(result) # this a boolean list of 'True' or 'False' if the card matches the valid vendor numbers 
        str_nums.append(str_num) # This is a list of all the transaction values as strings 
    results = pd.DataFrame({'credit_card_number':str_nums,
                            'ipv4':transaction_files['ipv4'],
                            'state':transaction_files['state'],
                            'valid_vendor':valid_nums})
    valid_results = results[results['valid_vendor'] == True]
    valid_results.loc[:,"start_val"] = start_vals    
    valid_results = valid_results.reset_index(drop=True)
    
    fraud_results = results[results['valid_vendor'] == False]
    fraud_results = fraud_results.reset_index(drop=True)
    return results, numbers, valid_results, fraud_results


results, numbers, valid_results, fraud_results = filter_transactions(transaction_files, credit_cards, start_vals)


In [ ]:
# valid_results
# fraud_results

In [1]:
import csv 
import json
import os 
import pandas as pd 
import sys 
from itertools import chain

In [61]:
from experiment import (get_fraud_file,
                        load_transaction_files, 
                        get_prefix_column,
                        filter_transactions,
                        filter_fraud_in_transactions,
                        fraud_by_state)

In [63]:
test_fraud_file1 = ('test_datasets/test_fraud')
test_file_list = ['test_datasets/transaction_test1','test_datasets/transaction_test2']
credit_cards = ['5018','5020','2131','56','300', '6011']

fraud_dataset1 = get_fraud_file(test_fraud_file1)
transaction_files = load_transaction_files(test_file_list)
start_vals = get_prefix_column(transaction_files, credit_cards)
results, numbers, valid_results, fraud_results = filter_transactions(transaction_files, credit_cards, start_vals)
valid_transactions, fraud_transactions = filter_fraud_in_transactions(fraud_dataset1, valid_results)
fraud_by_state = fraud_by_state(fraud_transactions)

TypeError: 'DataFrame' object is not callable

In [66]:
fraud_groupby_state = fraud_transactions.groupby('state').count()
fraud_by_state = fraud_groupby_state[['credit_card_number']]
fraud_by_state = fraud_by_state.rename(columns={'credit_card_number': 'count'})

In [72]:
valid_results

,credit_card_number,ipv4,state,valid_vendor,start_val
0,5018192281,1.22.333.4444,AZ,True,5018
1,5020843874,1.22.333.4444,CA,True,5020
2,2131829272,1.22.333.4444,TX,True,2131
3,5647382910,1.22.333.4444,NJ,True,56
4,3000000000,1.22.333.4444,TN,True,300
5,6011373939,1.22.333.4444,AL,True,6011
6,2131838932,1.22.333.4444,NM,True,2131


## Part II 

- Candidate should assume that going forward, only the sanitized dataset should be used.


### - Find in the sanitized dataset if it contains fraudulent transactions (using the data from loading fraud.zip) and report their number.


In [ ]:
def filter_fraud_in_transactions(fraud_dataset, valid_results):
    """
    From the santitized results dataframe 
    filter the DF to display only the numbers
    that are present in the fraud.zip file 
    """
#     This just produces a list of all the fraud credit_cards that exist in the sanitized dataframe
#     fraud_transactions = list(set(results_filtered['credit_card_number']).intersection(fraud_dataset))

#     A better way is to produce the whole dataframe and other columns of the credit_cardscredit_card_numbers in the sanitized
#     dataframe that also exist in fraud.zip 
    fraud_transactions = valid_results[valid_results['credit_card_number'].isin(fraud_dataset)].drop_duplicates()
    fraud_transactions = fraud_transactions.reset_index(drop=True)
    fraud_transactions['valid_card'] = False
    valid_transactions = valid_results.drop(fraud_transactions.index)
    valid_transactions['valid_card'] = True
    return valid_transactions, fraud_transactions

In [ ]:
valid_transactions, fraud_transactions = filter_fraud_in_transactions(fraud_dataset, valid_results)

In [ ]:
model_data = valid_transactions.append(fraud_transactions)
# model_data.to_csv('modeling_data.csv')

In [ ]:
print('Within the sanitized dataframe, there were', len(fraud_transactions), 'instances of fraudulent transactions.')

### - Create a report of the number of fraudulent transactions per state.

In [ ]:
# - Create a report of the number of fraudulent transactions per state.
### MAY WANT TO CONSIDER RETURNING A JSON FORMAT FILE 

def get_fraud_by_state(fraud_transactions):
    """
    Takes in a dataframe (fraud_transactions within valid_results)
    performs a group by on state,
    and returns a json serialized list of the count of
    transactions by state as objects
    """
    fraud_groupby_state = fraud_transactions.groupby('state').count()
    fraud_by_state = fraud_groupby_state[['credit_card_number']]
    fraud_by_state = fraud_by_state.rename(columns={'credit_card_number': 'count'})

    return fraud_by_state
fraud_by_state = get_fraud_by_state(fraud_transactions)

In [ ]:
# print(fraud_by_state.sum())
# fraud_by_state[0:10]

### - Create a report of the number of fraudulent transactions per card vendor, e.g., maestro => 45, amex => 78, etc..


In [ ]:
def get_vendors_val_key(item):
    for key, val in vendors.items():
        if item in val:
            return key

In [ ]:
# - Create a report of the number of fraudulent transactions per vendor.
### MAY WANT TO CONSIDER RETURNING A JSON FORMAT FILE 

def get_fraud_by_vendor(fraud_transactions):
    """
    Takes in an dataframe performs a group by on state,
    and returns a json serialized list of the count of
    transactions by state as objects
    """
    fraud_transactions['start_val']= fraud_transactions['start_val'].map(str)
    fraud_transactions['name'] = fraud_transactions.start_val.map(get_vendors_val_key)
    fraud_groupby_vendor = fraud_transactions.groupby('name').count()
    fraud_by_vendor = fraud_groupby_vendor[['credit_card_number']]
    fraud_by_vendor = fraud_by_vendor.rename(columns={'credit_card_number': 'count'})
    return fraud_transactions, fraud_by_vendor

In [ ]:
# fraud_transactions

In [ ]:
fraud_transactions, fraud_by_vendor = get_fraud_by_vendor(fraud_transactions)
print(fraud_by_vendor.sum())
# fraud_by_vendor

### - Create a dataset of 3 columns and save in both JSON and in a binary fileformat that you believe it's suitable for BI analysis:
####   - column 1: masked credit card: replace 9 last digits of the credit card with `*`
####   - column 2: ip address
####   - column 3: state
####   - column 4: sum of number of byte of (column 1 + column 2 + column 3)

In [ ]:
def masked_credit_cards(valid_results):
    dicts = []
    for index, row in enumerate(valid_results['credit_card_number']):
        full_card_number = valid_results['credit_card_number'].astype(str).iloc[index][:-9]
        masked = full_card_number + '*' * 9
        ipv4_value = valid_results['ipv4'][index]
        state_value = valid_results['state'][index]
        byte_number = sys.getsizeof(masked + ipv4_value + state_value)
        dict = {'credit_card_number' : masked,
                 'ipv4' : ipv4_value,
                 'state' : state_value,
                 'bytes' : byte_number}
        dicts.append(dict)
    df_dicts = pd.DataFrame(dicts) 
    return df_dicts

In [ ]:
# tmp = fraud_transactions[0:10]

In [ ]:
# %%time
# final_df = masked_credit_cards(valid_results)
# final_df

In [ ]:
final_df = masked_credit_cards(valid_results)
final_df

In [ ]:
def make_json_file(final_df):
    directory = 'results/'
    sample = final_df.to_dict()
    with open(directory + "final_df.json", "w") as outfile:
        json.dump(sample, outfile)

In [ ]:
make_json_file(final_df)

In [ ]:
def make_binary_file(final_df):
    directory = 'results/'
    with open(directory + 'sample', 'wb') as f:
        f.write(bytes(final_df, encoding='utf8'))
    

In [ ]:
make_binary_file(final_df)

In [ ]:
def generate_json_file(data, file):
    directory = 'results/'
    with open(directory + file + '.json', 'w') as f:
        f.write(data)

def generate_binary_file(data, file):
    directory = 'results/'
    with open(directory + file, 'wb') as f:
        f.write(bytes(data, encoding='utf8'))

### Part IV 

#### - Create a ML model that you think is useful using the datasets provided (raw or/and sanitized)
#### - Provide insight/metrics on how to validate if the model is accurate.

In [ ]:
# frauddata = ('/Users/sarahkaunitz/Documents/deloitte/AI_Engineer/code_challenge/datasets/modeling_data.csv')
# fraud_data = pd.read_csv(frauddata)
# fraud_data.drop(['Unnamed: 0'], axis=1)